In [1]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
# from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
def clean(text):
    """ Function to clean the text """
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    texter = re.sub(r"<br />", " ", text)
    texter = re.sub(r"&quot;", "\"",texter)
    texter = re.sub('&#39;', "\"", texter)
    texter = re.sub('\n', " ", texter)
    texter = re.sub(' u '," you ", texter)
    texter = re.sub('`',"", texter)
    texter = re.sub(' +', ' ', texter)
    texter = re.sub(r"(!)\1+", r"!", texter)
    texter = re.sub(r"(\?)\1+", r"?", texter)
    texter = re.sub('&amp;', 'and', texter)
    texter = re.sub('\r', ' ',texter)
    
    # Remove numbers from string
    texter = re.sub(pattern=r"[+-]?\d+(?:\.\d+)?", repl="", string=texter, count=0, flags=0)
    texter = texter.replace("  ", " ")
    clean = re.compile('<.*?>')
    texter = texter.encode('ascii', 'ignore').decode('ascii')
    texter = re.sub(clean, '', texter)
    texter = re.sub(r'[^\w\s]', '', texter)
    if texter == "":
        texter = ""
    return texter

In [4]:
glove_dir = './embeddings/glove.6B.300d.txt'
input_dim = 300

vocab = {}
with open(glove_dir, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        vocab[word] = vector
f.close()

print('Found %s word vectors.' %len(vocab))

Found 400000 word vectors.


In [5]:
max_len = 50
zero_padding = [0]*input_dim

def get_embeddings(text, emb="LSTM"):
    if emb == "DNN":
        embedding = [0]*input_dim
        i = 0
        for word in text.split(' '):
            if word in vocab:
                i += 1
                embedding += vocab[word]

        if i != 0:
            embedding /= i

#         scale = 0 # noise weight_scale
#         noise = np.random.randn(input_dim)*scale
#         embedding += noise
    elif emb == "LSTM":
        embedding = []
        i = 0
        for word in text.split(' '):
            if i == max_len:
                break
            if word in vocab:
                i += 1
                embedding.append(vocab[word])
        
        while i < max_len:
            i += 1
            embedding.append(zero_padding)
            
    return embedding

In [6]:
def transform(X, emb="LSTM"):
    embeddings = []
    for item in X:
        item = clean(item)
        embedding = get_embeddings(item, emb)
        embeddings.append(embedding)
    
    return embeddings

In [7]:
cleaned_X = np.array(transform(df['text']))
y = np.array(df['target'])

In [20]:
# Hyperparameters
num_epochs = 8
batch_size = 128
learning_rate = 0.00002
dropout = 0.25

In [21]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_X, y, test_size=0.15, random_state=42)

X_tr = torch.tensor(X_train, dtype=torch.float)
y_tr = torch.tensor(y_train)
train = TensorDataset(X_tr, y_tr)
trainloader = DataLoader(train, batch_size=batch_size)

X_te = torch.tensor(X_test, dtype=torch.float)
y_te = torch.tensor(y_test)
test = TensorDataset(X_te, y_te)
testloader = DataLoader(test)

In [22]:
class LSTM(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout):
        super(LSTM, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True, bidirectional=True) #lstm
        self.fc1 =  nn.Linear(hidden_size*seq_length*2, 128) #fully connected 1
        self.fc2 = nn.Linear(128, num_classes) #fully connected last layer
        
        self.dropout = nn.Dropout(dropout)
        self.batchnorm1 = nn.BatchNorm1d(128)

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size) #hidden state
        c_0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
        x, _ = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        
        # Flatten lstm output
        x = x.reshape(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.dropout(x)
#         x = F.log_softmax(self.fc2(x), dim=1)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

lstm = LSTM(2, 300, 100, 2, 50, dropout)

print(lstm)

LSTM(
  (lstm): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)
  (fc1): Linear(in_features=10000, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (batchnorm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [23]:
# Loss function
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

In [25]:
# Training
lstm.train()
training_loss = []
for epoch in range(num_epochs):
    lstm.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data

        outputs = lstm(inputs)
        loss = criterion(outputs, labels)
        training_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, loss.data))
        
    # Validation
    lstm.eval()
    outputs = lstm(X_te)
    _, predicted = torch.max(outputs, 1)
    total = y_te.size(0)
    correct = (predicted == y_te).sum()

    print(f'Accuracy of the model is: {100*correct/total:.2f}%')


Epoch [1/8], Loss: 0.5393
Accuracy of the model is: 80.30%
Epoch [2/8], Loss: 0.5327
Accuracy of the model is: 80.47%
Epoch [3/8], Loss: 0.5132
Accuracy of the model is: 80.47%
Epoch [4/8], Loss: 0.5172
Accuracy of the model is: 81.52%
Epoch [5/8], Loss: 0.5042
Accuracy of the model is: 81.44%


In [26]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
cleaned_X_test = np.array(transform(df_test['text']))
X_test = torch.tensor(cleaned_X_test, dtype=torch.float)

In [28]:
# Testing
lstm.eval()
outputs_test = lstm(X_test)

_, predicted_test = torch.max(outputs_test, 1)

In [29]:
data = {'id': np.array(df_test['id']),
       'target': np.array(predicted_test)}

In [30]:
df_submission = pd.DataFrame(data)
df_submission.to_csv('submission_lstm.csv', encoding='utf-8', index=False)